In [4]:
# 띄어쓰기 

!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
from pykospacing import spacing
text = spacing("이정도 우울증 증상 병원가봐야하나요?")
print(text)

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-63g8u3nw
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-63g8u3nw
  Created wheel for pykospacing: filename=pykospacing-0.3-cp36-none-any.whl size=2255638 sha256=0a29b36ccd6504f112711091be94a47e980bfa9e1f0e3d5f4e2a2217ce31a347
  Stored in directory: /tmp/pip-ephem-wheel-cache-l0br6ze_/wheels/4d/45/58/e26cb2b7f6a063d234158c6fd1e5700f6e15b99d67154340ba
Successfully built pykospacing


명사 중 빈도 수가 높은 것들 추출

In [10]:
!pip install konlpy
from konlpy.tag import Kkma
from re import match
from collections import Counter
import pandas as pd

# ----------------------- 변수 ----------------------------------------------------------
DATASET_PATH = '/content/drive/My Drive/Colab Notebooks/recommend_music_by_sentiment_analysis/dataset/'
EXCEL_NAME = 'sad.xlsx'
TOP_NUMBER = 10
# ----------------------------------------------------------------------------------------

kkma = Kkma()
xlsx = pd.read_excel(DATASET_PATH + EXCEL_NAME, names = ['c1', 'c2'])
docs = ""
for i in xlsx['c1']:
  docs+=i+'.'
ex_sent = kkma.sentences(docs)
nouns = []
for sent in ex_sent:
  for noun in kkma.nouns(sent):
    if len(str(noun)) >= 2 and not(match('^[0-9]', noun)):
      nouns.append(noun)

word_count = {}
for noun in nouns:
  word_count[noun] = word_count.get(noun, 0) + 1


counter = Counter(word_count)
top_words = counter.most_common(TOP_NUMBER)
top_words

[('때문', 236),
 ('눈물', 230),
 ('노래', 225),
 ('친구', 194),
 ('상해', 168),
 ('추천', 164),
 ('이유', 137),
 ('요즘', 84),
 ('주세', 79),
 ('남자', 68)]

word_cloud를 위한 한글 글씨체 다운로드

In [12]:
!apt-get install fonts-nanum*
!apt-get install fontconfig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'fonts-nanum-eco' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum-gothic-light' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum-coding' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum-extra' for glob 'fonts-nanum*'
fonts-nanum is already the newest version (20170925-1).
fonts-nanum-coding is already the newest version (2.5-1).
fonts-nanum-eco is already the newest version (1.000-6).
fonts-nanum-extra is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
fontconfig is already the newest version (2.12.6-0ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


wordcloud 생성

In [1]:
!pip install WordCloud
import sys
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud
import pandas as pd


# --------------------------------------- 변수 -------------------------------
DATASET_PATH = '/content/drive/My Drive/Colab Notebooks/recommend_music_by_sentiment_analysis/dataset/'
RESULT_PATH = '/content/drive/My Drive/Colab Notebooks/recommend_music_by_sentiment_analysis/result/'
FONT_PATH = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
EXCEL_NAME = 'angry.xlsx'
PNG_NAME = 'angry.png'
COMMON_NUMBER = 50
# 기쁨(0) 슬픔(1) 우울(2) 설렘(3) 화남(4)
EMOTION_LIST = ["happy", 'sad', 'depressed', 'love', 'angry']
# ----------------------------------------------------------------------------

# 엑셀의 문장들을 하나의 문장으로 합침
def make_docs(): 
  xlsx = pd.read_excel(DATASET_PATH + EXCEL_NAME, names = ['c1', 'c2'])
  docs = ""
  for i in xlsx['c1']:
    docs+=str(i)+'.'
  return docs

# 불용어 사전 구축
def make_stop_words():
  f = open(DATASET_PATH + 'stop_words_kr.txt','r')
  stop_words = []
  while True:
      line = f.readline()
      if not line: break
      stop_words.append(line[:-1])
  return stop_words

# 고빈도 명사 리스트 구축
def get_noun(news, stop_words):
  okt = Okt()
  noun_ = okt.nouns(news)
  noun = []
  for i in noun_:
    noun.append(i.replace(" ", ""))
  new_noun = []
  for i, v in enumerate(noun):
    if (len(v) >= 2) and (v not in stop_words):
      new_noun.append(v)
      
  count = Counter(new_noun)
  noun_list = count.most_common(COMMON_NUMBER)
  return noun_list

# wordcloud 생성
def visualize(noun_list):
  wc = WordCloud(font_path = FONT_PATH,
                 background_color = "white",
                 width = 1000,
                 height = 1000,
                 max_words = COMMON_NUMBER,
                 max_font_size = 300)
  wc.generate_from_frequencies(dict(noun_list))
  wc.to_file(RESULT_PATH + PNG_NAME)

for emotion in EMOTION_LIST:
  EXCEL_NAME = emotion + '.xlsx'
  PNG_NAME = emotion + '.png'
  docs = make_docs()
  stop_words = make_stop_words()
  noun_list = get_noun(docs, stop_words)
  visualize(noun_list)
  print('finish ' + emotion)

finish happy
finish sad
finish depressed
finish love
finish angry
